# EXAMPLE DATA

In [1]:
cashPosition = ['Long', 10]          # 'Long' or 'Short'; quantity
underlyingPosition = ['Long', 1]     # 'Long' or 'Short'; quantity
optPositions = [['Call', 60, -1],
                ['Put', 40, 1]]      # call or put; strike price; quantity


# FUNCTIONS

In [2]:
import numpy as np
import pandas as pd
import plotly.express as px

class Option(list):
    def add(self, security, quantity, strike=None):
        self.append(dict(security=security, quantity=quantity, strike=strike))

    def strikes(self):
        return [
            d["strike"] for d in self if d["security"] not in ["Underlying", "Cash"]
        ]

    def grid(self):
        strikes = self.strikes()
        maxstrike = 100 if len(strikes) == 0 else 1.5 * np.max(strikes)
        return np.linspace(0, maxstrike, 200)

    def value(self):
        grid = self.grid()
        value = np.zeros(len(grid))
        for x in self:
            if x["security"] == "Cash":
                value += x["quantity"]
            elif x["security"] == "Underlying":
                value += x["quantity"] * grid
            elif x["security"] == "Call":
                value += x["quantity"] * np.maximum(grid - x["strike"], 0)
            else:
                value += x["quantity"] * np.maximum(x["strike"] - grid, 0)
        return value

    def plot(self):
        df = pd.DataFrame(self.grid(), columns=["Underlying"])
        df["Portfolio"] = self.value()
        fig = px.line(df, x="Underlying", y="Portfolio")
        fig.layout.xaxis["title"] = "Underlying Price"
        fig.layout.yaxis["title"] = "Portfolio Value"
        fig.update_layout(xaxis_tickprefix="$", xaxis_tickformat=",.0f")
        fig.update_layout(yaxis_tickprefix="$", yaxis_tickformat=",.0f")
        return fig

# FIGURE

In [3]:
x = Option()

if cashPosition[0] == "Long":
    x.add(security="Cash", quantity=cashPosition[1])
elif cashPosition[0] == "Short":
    x.add(security="Cash", quantity=-cashPosition[1])

if underlyingPosition[0] == "Long":
    x.add(security="Underlying", quantity=underlyingPosition[1])
elif underlyingPosition[0] == "Short":
    x.add(security="Underlying", quantity=-underlyingPosition[1])

for opt in optPositions:
    x.add(security=opt[0], strike=opt[1], quantity=opt[2])

x.plot()